In [2]:
!python -m jupytools export -nb "16_densenet_tricks.ipynb" -o .
!mv densenet_tricks.py densenet121_with_tricks.py

Exported: 16_densenet_tricks.ipynb -> densenet_tricks.py
1 notebook(s) exported into folder: .


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# https://www.kaggle.com/tanlikesmath/rcic-fastai-starter

## Imports 

In [3]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from functools import reduce
from pdb import set_trace

import cv2 as cv
import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt

from catalyst.utils import get_one_hot
from imageio import imread
import pretrainedmodels
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from visdom import Visdom

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
from basedir import ROOT, NUM_CLASSES
from dataset import build_stats_index
from lookahead import Lookahead

In [4]:
#export
torch.set_default_tensor_type(torch.FloatTensor)

## Dataset Reader

In [5]:
#export
from augmentation import JoinChannels, SwapChannels, Resize, ToFloat, Rescale
from augmentation import VerticalFlip, HorizontalFlip, PixelStatsNorm, composer
from augmentation import UnconditionalTransform
from augmentation import AugmentedImages, bernoulli

In [6]:
#export
default_open_fn = imread  # PIL.Image.open

In [7]:
#export
class RxRxDataset(Dataset):
    
    def __init__(self, meta_df, img_dir, sites=(1, 2), channels=(1, 2, 3, 4, 5, 6),
                open_image=default_open_fn, n_classes=NUM_CLASSES, train=True,
                flip_v=0.1, flip_h=0.1, resize=512, norm=True, rescale=False,
                crop=False, hat=False, sigma_clip=False, label_smoothing=0.0):
        
        # data
        self.records = meta_df.to_records(index=False)
        self.img_dir = img_dir
        self.sites = sites
        self.channels = channels
        self.n = len(self.records)
        self.open_image = open_image
        self.n_classes = n_classes
        self.train = train
        
        # options
        self.flip_v = flip_v
        self.flip_h = flip_h
        self.resize = resize
        self.norm = norm
        self.rescale = rescale
        self.crop = crop
        self.hat = hat
        self.sigma_clip = sigma_clip
        self.label_smoothing = label_smoothing
    
    def __getitem__(self, index):
        samples = {f'site{site}': self._get_site_image(index, site) for site in self.sites}
        return samples
    
    def __len__(self):
        return self.n
    
    def _get_image_path(self, index, channel, site):
        r = self.records[index]
        exp, plate, well = r.experiment, r.plate, r.well
        subdir = 'train' if self.train else 'test'
        path = f'{self.img_dir}/{subdir}/{exp}/Plate{plate}/{well}_s{site}_w{channel}.png'
        return path
        
    def _get_site_image(self, index, site):
        paths = [self._get_image_path(index, ch, site) for ch in self.channels]
        images = [self.open_image(p) for p in paths]
        image = self._concat(images)
        image = self._augment(image)
        sample = self._wrap_with_meta(image, self.records[index])
        sample['site'] = site
        return sample
        
    def _wrap_with_meta(self, image, meta):
        if self.train:
            sirna = meta.sirna
            target = int(sirna)
            onehot = get_one_hot(target, num_classes=self.n_classes,
                                 smoothing=self.label_smoothing)
            return {'features': image, 'targets': target, 'targets_one_hot': onehot,
                    'id_code': meta.id_code}
        else:
            return {'features': image, 'id_code': meta.id_code}
        
    def _concat(self, images):
        try:
            img = np.stack(images)
        except (TypeError, ValueError) as e:
            print(f'Warning: cannot concatenate images! {e.__class__.__name__}: {e}')
            for filename, image in zip(paths, images):
                print(f'\tpath={filename}, size={image.size}')
            index = (index + 1) % len(self)
            print(f'Skipping instance {index} and trying another one...')
            return self[index]
        finally:
            for image in images:
                if hasattr(image, 'close'):
                    image.close()
        return img
                    
    def _augment(self, image):
        # OpenCV channels ordering
        image = image.transpose(1, 2, 0)  # W x H x C
        if self.resize:
            image = cv.resize(image, (self.resize, self.resize))
        if self.crop:
            assert isinstance(self.crop, int), 'If crop provided, it should be integer'
            assert self.crop < self.resize, 'Crop should be smaller than image size'
            shift = np.random.randint(0, self.resize - self.crop - 1)
            image = image[shift:(shift+self.crop), shift:(shift+self.crop), :]
        if self.flip_v:
            if bernoulli(self.flip_v) == 1:
                image = cv.flip(image, 0)
        if self.flip_h:
            if bernoulli(self.flip_h) == 1:
                image = cv.flip(image, 1)
        if self.hat:
            kernel = np.ones(self.hat, np.uint8)
            image = cv.morphologyEx(image, cv.MORPH_TOPHAT, self.hat)
        image = image.astype(np.float32)
        if self.rescale:
            image /= 255
        if self.norm:
            mean, std = image.mean(axis=(0, 1)), image.std(axis=(0, 1))
            image = (image - mean)/(std + 1e-8)
        if self.sigma_clip:
            image = np.clip(image, -self.sigma_clip, self.sigma_clip)
        image = image.transpose(2, 0, 1)  # C x W x H
        return image

## Pipeline Preparation

In [8]:
#export
from split import StratifiedSplit
splitter = StratifiedSplit()
trn_df, val_df = splitter(pd.read_csv(ROOT/'train.csv')) 
tst_df = pd.read_csv(ROOT/'test.csv')

In [9]:
trn_df = trn_df.head(100)
val_df = val_df.head(100)

In [10]:
#export
sz = 512

trn_ds = RxRxDataset(
    trn_df, ROOT,
    crop=384, resize=sz, hat=(128, 128), norm=True, sigma_clip=5.0, label_smoothing=0.1)

val_ds = RxRxDataset(
    val_df, ROOT,
    crop=384, resize=sz, hat=(128, 128), norm=True, sigma_clip=5.0, label_smoothing=None)

In [11]:
#export
def new_loader(ds, bs, drop_last=False, shuffle=True, num_workers=12):
    return DataLoader(ds, batch_size=bs, drop_last=drop_last, 
                      shuffle=shuffle, num_workers=num_workers)

## Model

In [12]:
#export
def densenet(name='densenet121', n_classes=NUM_CLASSES):
    model_fn = pretrainedmodels.__dict__[name]
    model = model_fn(num_classes=1000, pretrained='imagenet')
    new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
    conv0 = model.features.conv0.weight
    with torch.no_grad():
        new_conv.weight[:, :] = torch.stack([torch.mean(conv0, 1)]*6, dim=1)
    model.features.conv0 = new_conv
    return model

In [13]:
#export
from catalyst.contrib.modules import GlobalConcatPool2d
class DenseNet_TwoSites(nn.Module):
    def __init__(self, name, n_classes=NUM_CLASSES):
        super().__init__()
        
        base = densenet(name=name, n_classes=n_classes)
        feat_dim = base.last_linear.in_features
        
        self.base = base 
        self.pool = GlobalConcatPool2d()
        self.head = nn.Sequential(
            nn.Linear(feat_dim * 2, feat_dim * 2),
            nn.BatchNorm1d(feat_dim * 2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(feat_dim * 2, n_classes)
        )
        
    def forward(self, s1, s2):
        f1 = self.base.features(s1)
        f2 = self.base.features(s2)
        f_merged = self.pool(f1 + f2)
        out = self.head(f_merged.squeeze())
        return out

In [14]:
#export
def freeze_all(model):
    for name, child in model.named_children():
        print('Freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = False

In [15]:
#export
def unfreeze_all(model):
    for name, child in model.named_children():
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

In [16]:
#export
def unfreeze_layers(model, names):
    for name, child in model.named_children():
        if name not in names:
            continue
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

In [17]:
#export
from torch.optim.lr_scheduler import _LRScheduler
class CosineDecay(_LRScheduler):
    def __init__(self, optimizer, total_steps,
                 linear_start=0,
                 linear_frac=0.1, min_lr=1e-6,
                 last_epoch=-1):
        
        self.optimizer = optimizer
        self.total_steps = total_steps
        self.linear_start = linear_start
        self.linear_frac = linear_frac
        self.min_lr = min_lr
        self.linear_steps = total_steps * linear_frac
        self.cosine_steps = total_steps - self.linear_steps
        super().__init__(optimizer, last_epoch)
        
    def get_lr(self):
        step = self.last_epoch
        if step <= self.linear_steps:
            b = self.linear_start
            return [(step/self.linear_steps) * (base_lr - b) + b for base_lr in self.base_lrs]
        else:
            t = self.last_epoch - self.linear_steps
            T = self.cosine_steps
            return [self.min_lr + (base_lr - self.min_lr)*(1 + np.cos(t*np.pi/T))/2
                    for base_lr in self.base_lrs]

## Train

In [ ]:
#export
model = DenseNet_TwoSites('densenet121')
freeze_all(model)

In [ ]:
#export
from visdom import Visdom

In [ ]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [ ]:
#export
def create_loaders(batch_size, drop_last=False):
    trn_dl = new_loader(trn_ds, bs=batch_size, drop_last=drop_last, shuffle=True)
    val_dl = new_loader(val_ds, bs=batch_size, drop_last=drop_last, shuffle=False)
    return OrderedDict([('train', trn_dl), ('valid', val_dl)])

In [ ]:
#export
class Checkpoint:
    def __init__(self, output_dir):
        if os.path.exists(output_dir):
            print('Warning! Output folder already exists.')
        os.makedirs(output_dir, exist_ok=True)
        self.output_dir = output_dir
    
    def __call__(self, epoch, **objects):
        filename = os.path.join(self.output_dir, f'train.{epoch}.pth')
        checkpoint = {}
        for k, v in objects.items():
            if hasattr(v, 'state_dict'):
                v = v.state_dict()
            checkpoint[k] = v
        torch.save(checkpoint, filename)
        return filename

In [ ]:
#export
class LabelSmoothingLoss(nn.Module):
    def __init__(self, dim=-1):
        super().__init__()
        self.dim = dim

    def forward(self, preds, one_hot_target):
        preds = preds.log_softmax(dim=self.dim)
        return torch.mean(torch.sum(-one_hot_target * preds, dim=self.dim))

In [ ]:
#export
# loss_fn = nn.CrossEntropyLoss()
loss_fn = LabelSmoothingLoss()
device = torch.device('cuda:0')

In [ ]:
#export
epochs = 50
patience = 15

opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
opt = Lookahead(opt)
model = model.to(device)
rolling_loss = dict(train=RollingLoss(), valid=RollingLoss())
steps = dict(train=0, valid=0)

trials = 0
best_metric = -np.inf
history = []
stop = False

vis = Visdom(server='0.0.0.0', port=9091,
             username=os.environ['VISDOM_USERNAME'],
             password=os.environ['VISDOM_PASSWORD'])

# loaders = create_loaders(batch_size=7)

# sched = CosineDecay(opt, total_steps=len(loaders['train']) * epochs, 
#                     linear_start=1e-6, linear_frac=0.1, min_lr=3e-6)

loaders = create_loaders(batch_size=8, drop_last=True)

eta_min = 3e-6

global_step = 0

sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    opt, T_0=len(loaders['train']), T_mult=2, eta_min=eta_min
)

checkpoint = Checkpoint('densenet121_with_tricks')

for epoch in range(1, epochs+1):
    print(f'Epoch [{epoch}/{epochs}]')
    
    if epoch == 1:
        unfreeze_layers(model, ['head'])
    elif epoch == 8:
        unfreeze_all(model)
        sched = CosineDecay(
            opt, total_steps=len(loaders['train']) * (epochs - 7), 
            linear_start=eta_min, linear_frac=0.1, min_lr=3e-6)
        global_step = 0
    
    iteration = dict(epoch=epoch, train_loss=list(), valid_loss=list())
    
    for name, loader in loaders.items():
        is_training = name == 'train'
        count = 0
        metric = 0.0
        
        with torch.set_grad_enabled(is_training):
            for batch_no, batch in enumerate(loader):
                steps[name] += 1
                opt.zero_grad()

                # y = batch['site1']['targets'].to(device)
                y = batch['site1']['targets_one_hot'].to(device)
                
                out = model(
                    batch['site1']['features'].to(device),
                    batch['site2']['features'].to(device)
                )
                
                if is_training:
                    global_step += 1
                    loss = loss_fn(out, y)
                    loss.backward()
                    opt.step()
                    
                    if (epoch == 7) and batch_no == (len(loader) - 1):
                        pass  # skip
                    else:
                        sched.step(global_step)
                    
                    curr_lr = opt.param_groups[0]['lr']
                    vis.line(X=[steps[name]], Y=[curr_lr], win='lr', name='lr', update='append')    
                
                avg_loss = rolling_loss[name](loss.item(), steps[name])
                iteration[f'{name}_loss'].append(avg_loss)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                y_true = batch['site1']['targets'].to(device)
                acc = (y_pred == y_true).float().mean().item()
                metric += acc
                count += len(batch)
                vis.line(X=[steps[name]], Y=[avg_loss], name=f'{name}_loss', 
                         win=f'{name}_loss', update='append', 
                         opts=dict(title=f'Running Loss [{name}]'))
        
        metric /= count
        iteration[f'{name}_acc'] = metric
        vis.line(X=[epoch], Y=[avg_loss], name=f'{name}', win='avg_loss',
                 update='append', opts=dict(title='Average Epoch Loss'))
        vis.line(X=[epoch], Y=[metric], name=f'{name}', win='accuracy', 
                 update='append', opts=dict(title=f'Accuracy'))
        
        last_loss = iteration[f'{name}_loss'][-1]
        
        print(f'{name} metrics: accuracy={metric:2.3%}, loss={last_loss:.4f}')
          
        if is_training:
            pass
          
        else:
            if metric > best_metric:
                trials = 0
                best_metric = metric
                print('Score improved!')
                checkpoint(epoch, model=model, opt=opt)

            else:
                trials += 1
                if trials >= patience:
                    stop = True
                    break
    
    history.append(iteration)
    
    print('-' * 80)
    
    if stop:
        print(f'Early stopping on epoch: {epoch}')
        break

torch.save(history, f'{checkpoint.output_dir}/history.pth')

## Test

In [18]:
device = torch.device('cuda:0')
model = DenseNet_TwoSites('densenet121')
model = model.to(device)
state = torch.load('densenet121_with_tricks/train.30.pth', map_location=lambda l, s: l)
model.load_state_dict(state['model'])
freeze_all(model)
_ = model.eval()

Freezing layer: base
Freezing layer: pool
Freezing layer: head


In [22]:
tst_ds = RxRxDataset(
    tst_df, ROOT,
    crop=384, resize=sz, hat=(128, 128), norm=True, sigma_clip=5.0, 
    label_smoothing=None, train=False)

In [23]:
with torch.no_grad():
    test_dl = new_loader(tst_ds, shuffle=False, bs=64)
    probs = {}
    for batch in tqdm(test_dl):
        s1 = batch['site1']['features']
        s2 = batch['site2']['features']
        out = model(s1.to(device), s2.to(device))
        y_prob = out.softmax(dim=-1).cpu().numpy()
        probs.update(dict(zip(batch['site1']['id_code'], y_prob)))

In [24]:
(pd.DataFrame(probs)
 .T
 .reset_index()
 .rename(columns={'index': 'id_code'})
 .to_csv('stacked/test/densenet121_with_tricks.csv', index=False))

In [25]:
sup_df = pd.read_csv(ROOT/'train.csv')

trn_ds = RxRxDataset(
    sup_df, ROOT,
    crop=384, resize=sz, hat=(128, 128), norm=True, sigma_clip=5.0, label_smoothing=0.1)

with torch.no_grad():
    test_dl = new_loader(trn_ds, shuffle=False, bs=64)
    probs = {}
    for batch in tqdm(test_dl):
        s1 = batch['site1']['features']
        s2 = batch['site2']['features']
        out = model(s1.to(device), s2.to(device))
        y_prob = out.softmax(dim=-1).cpu().numpy()
        probs.update(dict(zip(batch['site1']['id_code'], y_prob)))

In [26]:
(pd.DataFrame(probs)
 .T
 .reset_index()
 .rename(columns={'index': 'id_code'})
 .to_csv('stacked/train/densenet121_with_tricks.csv', index=False))

In [20]:
# sz = 512

# # TTA
# tst_ds = RxRxDataset(
#     tst_df, ROOT,
#     crop=384, resize=sz, hat=(128, 128), norm=True, sigma_clip=5.0, 
#     label_smoothing=None, train=False)

# tta = []

# for i in range(1, 6+1):
#     print(f'Round #{i}')
    
#     with torch.no_grad():
#         test_dl = new_loader(tst_ds, shuffle=False, bs=64)
#         preds = {}
#         for batch in tqdm(test_dl):
#             s1 = batch['site1']['features']
#             s2 = batch['site2']['features']
#             out = model(s1.to(device), s2.to(device))
#             y_hat = out.argmax(dim=-1).cpu().numpy()
#             preds.update(dict(zip(batch['site1']['id_code'], y_hat.astype(int))))
            
#     preds_df = pd.DataFrame([
#         {'id_code': id_code, 'sirna': sirna} 
#         for id_code, sirna in preds.items()])
    
#     tta.append(preds_df)

In [24]:
preds_df = pd.DataFrame([
    {'id_code': id_code, 'sirna': sirna} 
    for id_code, sirna in preds.items()])
preds_df.head(5)

,id_code,sirna
0,HEPG2-08_1_B03,855
1,HEPG2-08_1_B04,25
2,HEPG2-08_1_B05,650
3,HEPG2-08_1_B06,894
4,HEPG2-08_1_B07,878


In [25]:
len(preds_df)

19897

In [26]:
filename = 'densenet121_with_tricks.csv'
preds_df.to_csv(filename, index=False, columns=['id_code', 'sirna'])
from IPython.display import FileLink
FileLink(filename)

/home/ck/code/tasks/protein/densenet121_with_tricks.csv